# point_defect_formation_static
- - -

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2016-10-07

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## 1. Introduction

The __point_defect_formation_static__ calculation evaluates the formation energy for a variety of point defects and evaluates whether the defect configurations remain stable after relaxation. A perfect system is constructed and the cohesive energy is calculated. Then, a point defect is inserted into the system and the defect's formation energy is computed. The configuration of the defect and neighboring atoms of the relaxed system are used to check if the defect strucutre has relaxed to a lower energy configuration.

__Disclaimer__: this method uses pre-defined point defect configurations and relaxes the systems through energy/force minimization. The lowest energy defect configuration may not be one of the pre-defined ones and may require a more thorough (temperature) annealing to obtain. The algorithms used to identify when a defect structure relaxes to another configuration have been tested for simple materials/defects and may not work for all cases.

## 2. Calculation Input File Parameters

The calculation's input file expects certain key terms.

### 2.1 Commands

Provides the external commands for running LAMMPS and MPI.

- __lammps_command__: the path to the executable for running LAMMPS on your system. Required.

- __mpi_command__: the command associated with calling LAMMPS to run in parallel on your system. Default value is None (run serially).

### 2.2 Potential

Provides the information associated with an interatomic potential implemented for LAMMPS.

- __potential_file__: the path to the LAMMPS-potential data model used by atomman to generate the proper LAMMPS commands for an interatomic potential. Required.
 
- __potential_dir__: the path to the directory containing any potential artifacts (eg. eam setfl files) that are used. Default is '' (working directory for the simulation.)

### 2.3 System Load

Provides the information associated with loading an atomic configuration.

- __load__: the style and path to the initial configuration file being read in. The style can be any file type supported by atomman.System.load(). Required.
 
- __load_options__: a list of key-value pairs for the optional arguments in atomman.System.load().
 
- __symbols__: a space-delimited list of atom-model symbols corresponding to the atom types and potential. If not given, the element/symbol information in the loaded file will be used. Required if the load file does not contain element/symbol information. 
 
- __box_parameters__: box parameters to scale the loaded system to. If not given, the box parameters of the loaded file are used. 
    
    - three floats: a b c.  Allows for the definintion of orthorhombic lattice parameters.
    
    - six floats: a b c alpha beta gamma.  Allows for the definition of the lattice parameters and angles.

__NOTE__: The absolute box_parameter a,b,c values don't matter for the E_vs_r_scan calculation, but the b/a and c/a ratios and angles will affect the calculation.
    
### 2.4 System Manipulations

Performs manipulations on the loaded system. 

- __x-axis, y-axis, z-axis__: optional transformation axes for rotating the system. Each vector is given by three space-delimited numbers.  The vectors must be orthogonal to each other.  If the loaded system is cubic these vectors are taken as hkl crystallographic directions and the rotated system is transformed into an orthorhombic box with dimensions given by a\*sqrt(h<sup>2</sup>+k<sup>2</sup>+l<sup>2</sup>) for each axis. 

- __shift__: optional positional shift that is applied to all atoms.  The shift is applied after axis rotation, but before size_mults supersizing.

- __shift_scale__: Boolean flag indicating if the shift is absolute (False), or should be scaled relative to the system's unit cell (True).

- __size_mults__: optional parameters for supersizing the system. This may either be a list of three or six integers.

    - Three integers: mx my mz. The absolute values indicate how many times the system is multiplied along a given box axes and the sign indicates the direction.  
    
    - Six integers: nx px ny py nz pz. This allows for multiplications in both directions for each box axis. The n terms must be less than or equal to zero, and the p terms greater than or equal to zero. 
    
### 2.5 Defect Parameters

Specifies the parameters associated with adding a point defect to the system. The defect parameters can either be given using point_defect_model, or by giving the other point_defect parameters separately.

- __point_defect_model__: the path to a point defect data model file containing all necessary defect parameters.

- __point_defect_type__: indicates the type of point defect to add.
    - = __v__ or __vacancy__ for vacancy (removes an atom).
    - = __i__ or __interstitial__ for interstitial (adds an atom at a specified location).
    - = __s__ or __substitutional__ for substitutional (change atom type of specified atom).
    - = __d__ or __db__ or __dumbbell__ for dumbbell interstitial (replace single atom with a pair of atoms).

- __point_defect_atype__: indicates the atom type of the defect atom. Not used for vacancies. Default value is 1.

- __point_defect_pos__: indicates the position to add the defect. Alternative to point_defect_aid.

- __point_defect_aid__: indicates the atom id where defect is added. Not used for interstitials. Alternative to point_defect_pos.

- __point_defect_dumbbell_vect__: indicates the dumbbell vector to use. Only used for dumbbells 

- __point_defect_scale__: indicates if point_defect_pos and point_defect_dumbbell_vect values are in absolute (False) or box-relative (True) coordinates. Default is False.

### 2.6 Units

Specifies the units for various physical quantities to use in reading/writing values.

- __length_unit__: defines the unit of length for results, and input parameters if not specified. Default is 'angstrom'.

- __energy_unit__: defines the unit of energy for results, and input parameters if not specified. Default is 'eV'.

- __pressure_unit__: defines the unit of pressure for results, and input parameters if not specified. Default is 'GPa'.

- __force_unit__: defines the unit of pressure for results, and input parameters if not specified. Default is 'eV/angstrom'.

### 2.7 Run Parameters

Provides parameters specific to the calculation at hand.

- __energy_tolerance__: specifies the LAMMPS minimization energy tolerance (etol) to use for relaxing the system. Unitless. Default value is 0.0.     

- __force_tolerance__: specifies the LAMMPS minimization force tolerance (ftol) to use for relaxing the system. In LAMMPS units of force. Default value is 1e-6 eV/angstrom.     

- __maximum_iterations__: specifies the LAMMPS minimization maximum number of iterations (maxiter) to use for relaxing the system. Default value is 100000.     

- __maximum_evaluations__: specifies the LAMMPS minimization maximum number of evaluations (maxeval) to use for relaxing the system. Default value is 100000.  

## 3. Prepare Function Parameters

The input parameters recognized by the calculation's prepare function correspond to the calculation file input parameters with a few exceptions. Each parameter term can be specified with a list of values, with certain limitations placed on some parameters.

### 3.1 Additional terms:

These are additional terms that affect how the calculations are prepared.

- __run_directory__: the path to the directory where the calculation instances are being prepared. Required.

- __lib_directory__: the path to the directory where all XML results records are being stored. Required.

- __copy_files__: Boolean indicating if all required files for the calculation are copied into the calculation instance's folder or only the calculation script and input file. __Note__: runner.py currently requires copy_files to be True. Default value is True.

- __load_elements__: List of elements to use with the loaded system. User-specified wildcards are allowed. One or more symbols lists are generated from this based on the potential used.

### 3.2 Unused terms:

These are terms allowed by the calculation that are ignored by the prepare function.

- __x-axis, y-axis, z-axis__: complicates identifying where to add defects, and ideally doesn't affect results.

- __shift__: complicates identifying where to add defects, and ideally doesn't affect results.

- __symbols__: generated using load_elements.

- __point_defect_type, point_defect_atype, point_defect_pos, point_defect_aid, point_defect_dumbbell_vect, point_defect_scale__: use point_defect_models instead as they represent complete parameter sets.

### 3.3 Limitations

- __run_directory, lib_directory, copy_files, lammps_command, mpi_command, length_unit, pressure_unit, energy_unit, force_unit, energy_tolerance, force_tolerance, maximum_iterations, maximum_evaluations__: are limited to single values. 

- __potential_file, potential_dir__: must have the same number of values.

- __load, load_options, load_elements, box_parameters__: must have the same number of values.

- __point_defect_model__: at least one model must be specified.